# AGENT LANGCHAIN

## Tools

In [41]:
"""AgentExecutor
#STUDENT HOMEWORK
===============
Bob's birthday party; Bob's eleventh birthday party, to start the summer together!!; 2022-07-30; 16.00; 21.00; Turin, Bolin st., 65B
===============

#EVALUATION:
OK


#STUDENT HOMEWORK
===============
Explore the Universe; Learn about the latest discovery about the Universe's structure with prof. Delacour; 15:00; 17:30; University of Chicago, room 260, Chicago
===============

#EVALUATION:
The date of the event is missing."""

"AgentExecutor\n#STUDENT HOMEWORK\n===============\nBob's birthday party; Bob's eleventh birthday party, to start the summer together!!; 2022-07-30; 16.00; 21.00; Turin, Bolin st., 65B\n===============\n\n#EVALUATION:\nOK\n\n\n#STUDENT HOMEWORK\n===============\nExplore the Universe; Learn about the latest discovery about the Universe's structure with prof. Delacour; 15:00; 17:30; University of Chicago, room 260, Chicago\n===============\n\n#EVALUATION:\nThe date of the event is missing."

In [42]:
CHECK_INFO_PROMPT = """You are an extremely precise manager.
Your task is to check whether the business analyst of your team provided all the required information.
The job the business analyst had was to retrieve some data about an event, including:
- event name
- event description
- start and end date and time
- adress (composed of City and Adress)

If the business analyst provided all the info listed above, you must answer "OK" AND NOTHING ELSE.
Otherwise, you must specify the info that is missing. Be precise and CONCISE in your answer. List all the items that are missing.


#BUSINESS ANALYST RESULT
===============
{input}
===============

#EVALUATION:
"""

In [43]:

from langchain_aws import ChatBedrock
from langchain.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

check_info_chain = (
    PromptTemplate.from_template(CHECK_INFO_PROMPT)
    | ChatBedrock(
        model_id="mistral.mixtral-8x7b-instruct-v0:1",
        region_name="us-east-1",
    ) 
    | StrOutputParser()
)

In [44]:

from langchain.agents import  tool

@tool
def create_event(specifics:str):
    """Creates the event. Syntax: event name; event description; date; start time; ending time; adress."""
    splitted = specifics.split(";")
    check_info_chain = (
        PromptTemplate.from_template(CHECK_INFO_PROMPT)
        | ChatBedrock(
            model_id="mistral.mixtral-8x7b-instruct-v0:1",
            region_name="us-east-1",
        ) 
        | StrOutputParser()
    )
    missing = check_info_chain.invoke({"input":specifics})
    if missing.strip() == "OK":
        print(f"Create event with: {splitted}")
        return "Event created"
    else:
        return "Failed to create the event:" + missing + "\nPlease ask the user the missing information."
    
    
tools = [create_event]


In [45]:
check_info_chain.invoke({"input":"Giochiamo a WarHammer; Torneo di warhammer; [data]; [ora di inizio]; [ora di fine]; Via dei pini 7, Verona"}, config={"stop":"['\n']"})

' Missing information: event description, start date, end date.\n\nThe provided information is: "Giochiamo a WarHammer; Torneo di warhammer; [data]; [ora di inizio]; [ora di fine]; Via dei pini 7, Verona". It includes the event name ("Giochiamo a WarHammer; Torneo di warhammer"), but not the event description, start date, or end date. It also includes the address "Via dei pini 7, Verona", but this should be broken down into city and address for the purpose of this evaluation.'

## Prompt

In [46]:
EVENT_PROMPT = """You are a powerful event manager who works for Ludus Gate, a board game association.
Your task is to create events based on the user's specifics.
In order to successfully create an event you need to know the following pieces of information:
- what the event is about, that is, event name and a brief description;
- when the event takes place, that is, day (or possibly days) and start and ending time;
- where the event takes place, that is, city and adress.

You must ask the user for this information before creating the event.
Note that you can create the event only if you have **ALL** the information listed above. So always make sure the user specified everithing.
Below is the conversation so far. Please always answer in Italian.
"""

In [47]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system = '''Assistant is a large language model trained by Mistral.

Assistant is designed to be able to assist LudusGate, a board game association, with \
the creation of events. As a language model, Assistant is able to generate human-like \
text based on the input it receives, allowing it to engage in natural-sounding \
conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly \
evolving. It is able to process and understand large amounts of text, and can use this \
knowledge to provide accurate and informative responses to a wide range of questions. \
Additionally, Assistant is able to generate its own text based on the input it \
receives, allowing it to engage in discussions and provide explanations and \
descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks \
and provide valuable insights and information on a wide range of topics. Whether \
you need help with a specific question or just want to have a conversation about \
a particular topic, Assistant is here to assist.'''

human = '''TOOLS
------
Assistant can ask the user to use tools to create the event. \
Before using any of the tools, please make sure the user specified all the necessary information.
The tools the human can use are:

{tools}

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of three formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string, \ The action to take. Must be one of {tool_names}
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human or TO ASK ADDITIONAL INFO. Markdown code snippet formatted \
in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put what you want to say to the user here (in ITALIAN)
}}
```

USER'S INPUT
--------------------
Here is the user's input (remember to respond with a markdown code snippet of a json \
blob with a single action, and NOTHING else):

{input}'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

## Model

In [48]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="mistral.mixtral-8x7b-instruct-v0:1",
    region_name="us-east-1",
)

## Agent

In [49]:
from langchain.agents import AgentExecutor, create_json_chat_agent

agent = create_json_chat_agent(llm, tools, prompt, stop_sequence=False)

In [2]:
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory(session_id="test-session")

messages = [
    {"author":"user", "content":"Ciao, devo creare un evento per un torneo di warhammer", "timestamp":"1"},
    {"author":"bot", "content":"Ciao! Per creare l'evento per il torneo di Warhammer, ho bisogno di alcune informazioni aggiuntive. Mi servirebbe il nome dell'evento, una descrizione, la data, l'ora di inizio e di fine, e l'indirizzo dove si svolgerà. Puoi fornirmi queste informazioni?", "timestamp":"2"},
    {"author":"user", "content":"l'evento sarà a verona, in via dei pini 7", "timestamp":"3"},
    {"author":"bot", "content":"Grazie per l'informazione. Il torneo di Warhammer si svolgerà a Verona, in via dei pini 7. Per creare l'evento, ho ancora bisogno di alcune informazioni: il nome dell'evento, una descrizione, la data, e l'ora di inizio e di fine. Puoi fornirmi queste informazioni?", "timestamp":"4"},
    {"author":"user", "content":"l'evento si chiama 'Giochiamo a WarHammer', come descrizione tieni pure Torneo di warhammer", "timestamp":"5"},
    {"author":"bot", "content":"Mi dispiace, ma per creare l'evento ho bisogno anche della data, dell'ora di fine e della città dove si svolgerà il torneo di Warhammer chiamato 'Giochiamo a WarHammer'. Puoi fornirmeli?", "timestamp":"6"},
]
messages.sort(key= lambda x:x["timestamp"])
for msg in messages:
    if msg["author"] == 'bot':
        memory.add_ai_message(msg["content"])
    else:
        memory.add_user_message(msg["content"])

In [51]:
from langchain_core.runnables.history import RunnableWithMessageHistory


agent_executor = AgentExecutor(agent=agent, tools=tools, return_intermediate_steps=True, verbose = True)


In [52]:
import langchain
langchain.debug=True

In [53]:
agent_executor.invoke(
    {
        "input":"sarà il 13 maggio 2023 dalle 10 alle 18",
        "chat_history": memory.messages
    },
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
[inputs]
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [chain:AgentExecutor 

{'input': 'sarà il 13 maggio 2023 dalle 10 alle 18',
 'chat_history': [HumanMessage(content='Ciao, devo creare un evento per un torneo di warhammer'),
  AIMessage(content="Ciao! Per creare l'evento per il torneo di Warhammer, ho bisogno di alcune informazioni aggiuntive. Mi servirebbe il nome dell'evento, una descrizione, la data, l'ora di inizio e di fine, e l'indirizzo dove si svolgerà. Puoi fornirmi queste informazioni?"),
  HumanMessage(content="l'evento sarà a verona, in via dei pini 7"),
  AIMessage(content="Grazie per l'informazione. Il torneo di Warhammer si svolgerà a Verona, in via dei pini 7. Per creare l'evento, ho ancora bisogno di alcune informazioni: il nome dell'evento, una descrizione, la data, e l'ora di inizio e di fine. Puoi fornirmi queste informazioni?"),
  HumanMessage(content="l'evento si chiama 'Giochiamo a WarHammer', come descrizione tieni pure Torneo di warhammer"),
  AIMessage(content="Mi dispiace, ma per creare l'evento ho bisogno anche della data, dell'or

In [3]:
from src.event_handler import EventHandler

bot = EventHandler(chat_history=messages)

In [5]:
bot.run(message="sarà il 13 maggio 2023 dalle 10 alle 18")



> Entering new AgentExecutor chain...
 ```json
{
    "action": "create_event",
    "action_input": "Giochiamo a WarHammer; Torneo di WarHammer; 13 maggio 2023; 10:00; 18:00; Verona, via dei pini 7"
}
```Create event with: ['Giochiamo a WarHammer', ' Torneo di WarHammer', ' 13 maggio 2023', ' 10:00', ' 18:00', ' Verona, via dei pini 7']
Event created ```json
{
    "action": "Final Answer",
    "action_input": "L'evento 'Giochiamo a WarHammer' si svolgerà il 13 maggio 2023 a Verona, in via dei pini 7, dalle 10:00 alle 18:00. Il torneo di WarHammer è stato creato con successo."
}
```

> Finished chain.


{'input': 'sarà il 13 maggio 2023 dalle 10 alle 18',
 'chat_history': [AIMessage(content='Ciao, devo creare un evento per un torneo di warhammer'),
  HumanMessage(content="Ciao! Per creare l'evento per il torneo di Warhammer, ho bisogno di alcune informazioni aggiuntive. Mi servirebbe il nome dell'evento, una descrizione, la data, l'ora di inizio e di fine, e l'indirizzo dove si svolgerà. Puoi fornirmi queste informazioni?"),
  AIMessage(content="l'evento sarà a verona, in via dei pini 7"),
  HumanMessage(content="Grazie per l'informazione. Il torneo di Warhammer si svolgerà a Verona, in via dei pini 7. Per creare l'evento, ho ancora bisogno di alcune informazioni: il nome dell'evento, una descrizione, la data, e l'ora di inizio e di fine. Puoi fornirmi queste informazioni?"),
  AIMessage(content="l'evento si chiama 'Giochiamo a WarHammer', come descrizione tieni pure Torneo di warhammer"),
  HumanMessage(content="Mi dispiace, ma per creare l'evento ho bisogno anche della data, dell'or